In [176]:
from math import log10, fabs

import pandas as pd
import numpy as np
from PIL import Image

def check_pixel_colour(pixel, colour, delta=(0,0,0)):
    '''сравниваем цвет пиксиля с эталонным цветом с погрешностью дельта'''
    return (colour[0] - delta[0]) <= pixel[0] <= (colour[0] + delta[0]) and\
           (colour[1] - delta[1]) <= pixel[1] <= (colour[1] + delta[1]) and\
           (colour[2] - delta[2]) <= pixel[2] <= (colour[2] + delta[2])


def better_colour_value(pixels, colour):
    '''из списка цветов выбираем более подходящий эталонному цвету'''
    delta = 1024
    better_pixel = None
    for pixel in pixels:
        current_delta = fabs(pixel[0] - colour[0]) +\
                        fabs(pixel[1] - colour[1]) +\
                        fabs(pixel[2] - colour[2])
        if current_delta < delta:
            delta = current_delta
            better_pixel = pixel
    return better_pixel


def get_border_grids(image, grid_colour):
    '''
    получаем номера пиксилей по оси x первой и последней линий сетки, по оси y верхней и нижней
    данные пиксили соответствуют переданным граничным значениям сетки
    '''
    width, height = image.size
    x1, x2, y1, y2 = 0 , width , 0 , height
    for x in range(width):
        if check_pixel_colour(image.getpixel((x, 0)), grid_colour):
            x1 = x
            break
    for x in range(width - 1, -1, -1):
        if check_pixel_colour(image.getpixel((x, 0)), grid_colour):
            x2 = x
            break
    for y in range(height - 1, -1, -1):
        if check_pixel_colour(image.getpixel((0, y)), grid_colour):
            y1 = y
            break
    for y in range(height):
        if check_pixel_colour(image.getpixel((0, y)), grid_colour):
            y2 = y
            break
    return x1, x2, y1, y2

def get_linear_value(x, x1, x2, x1_value, x2_value):
    '''получаем значение для текущего пикселя с учетом граничных значений сетки для линейной оси'''
    k = (x2 - x1) / (x2_value - x1_value)
    b = x1 - k * x1_value
    return (x - b) / k


def get_log_value(x, x1, x2, x1_value, x2_value):
    '''получаем значение для текущего пикселя с учетом граничных значений сетки для логорифмической оси'''
    k = (x1 - x2) / (log10(x1_value) - log10(x2_value))
    b = x1 - k * log10(x1_value)
    return 10**((x - b) / k)


def process_graph(image_path, x1_value, x2_value, y1_value, y2_value, colour, delta, grid_colour):
    '''получаем список координат графика на изображении'''
    image = Image.open(image_path)
    width, height = image.size
    x1, x2, y1, y2 = get_border_grids(image, grid_colour)
    y_mean = 0
    coordinates = []
    y_list = []
    for x in range(width):
        pixels = {}
        for y in range(height):
            pixel = image.getpixel((x, y))
            if check_pixel_colour(pixel, colour, delta):
                pixels[pixel] = y
        if len(pixels) > 0:
            y = pixels[better_colour_value(pixels.keys(), colour)]
            x_value = get_linear_value(x, x1, x2, x1_value, x2_value)
            y_value = get_linear_value(height - y, y1, y2, y1_value, y2_value)
            if(x>6):
                y_mean = np.mean(y_list[-5:-1])
            else:
                coordinates.append((x_value, y_value))
                y_list.append(y)
            if(abs(y-y_mean)<10):
                coordinates.append((x_value, y_value))
                y_list.append(y)

    return coordinates

In [178]:
fon_val = 256
fon = (fon_val,fon_val,fon_val)
line_val = 30
line = (line_val,line_val,line_val)
grid_val = 256
grid = (grid_val,grid_val,grid_val)

df = pd.DataFrame(process_graph('graph1.jpg', 4.1, 7.6, 0, 1.9, line, fon, grid),columns=['time','Mau'])

df.shape

import plotly.express as px

fig = px.line(df, x="time", y="Mau", title='Хроматограмма')
fig.show()

In [44]:
fon_val = 256
fon = (fon_val,fon_val,fon_val)
line_val = 30
line = (line_val,line_val,line_val)
grid_val = 256
grid = (grid_val,grid_val,grid_val)

df = pd.DataFrame(process_graph('graph1.jpg', 4.1, 7.6, 0, 1.9, line, fon, grid),columns=['time','Mau'])

df.shape

import plotly.express as px

fig = px.line(df, x="time", y="Mau", title='Хроматограмма')
fig.show()

In [218]:
fon_val = 256
fon = (fon_val,fon_val,fon_val)
line_val = 0
line = (line_val,line_val,line_val)
grid_val = 256
grid = (grid_val,grid_val,grid_val)

df = pd.DataFrame(process_graph('graph2.jpg', 4.5, 8.5, 0, 20, line, fon, grid),columns=['time','Mau'])

df.shape

import plotly.express as px

type_name = 'AT1'
df[type_name] = 'до очистки'
df['Mau'] = df['Mau'] - 1
df1= df.copy()
df1['Mau'] = df1['Mau']/16*1.9
df1[type_name] = 'после очистки'
df1 = df1.drop(index=df1.query('time>=6.5').index)

df2 = df1.query('time>=4.5 and time<6.5')
df2['time'] = df2['time'] + 2
df2['Mau'] = df2['Mau'] + 0.71

df3 = pd.concat([df1,df2])


fig = px.line(pd.concat([df,df3]), x="time", y="Mau",color = type_name, title='Хроматограмма')
fig.show()